In [19]:
# Import Dependencies
import pandas as pd
import numpy as np

from config import postgres_password
from config import postgres_port

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
base = declarative_base

In [5]:
def get_cel_value(df, col_name):
    return df.at[0,col_name]

In [6]:
# Make a reference to the books.csv file path
entpre_acq_file = "../Resources/Census_ASE1/ASE_2016_00CSCBO01.csv"

# Import the books.csv file as a DataFrame
entpre_acq_dirty_df = pd.read_csv(entpre_acq_file, encoding="utf-8")
entpre_acq_dirty_df.head(2)

,GEO.id,GEO.id2,GEO.display-label,GEO.annotation.id,NAICS.id,NAICS.display-label,NAICS.annotation.id,ASECBO.id,ASECBO.display-label,YIBSZFI.id,YIBSZFI.display-label,ACQBUS.id,ACQBUS.display-label,YEAR.id,OWNPDEMP,OWNPDEMP_PCT,OWNPDEMP_S,OWNPDEMP_PCT_S
0,Geographic identifier code,Id2,Geographic area name,Id,2012 NAICS code,Meaning of 2012 NAICS code,Id,"Gender, ethnicity, race, and veteran status code","Meaning of Gender, ethnicity, race, and vetera...",Years in business code,Meaning of Years in business code,How the owner initially acquired the business ...,Meaning of How the owner initially acquired th...,Year,Number of owners of respondent firms with paid...,Percent of number of owners of respondent firm...,Relative standard error of number of owners of...,Standard error of percent of number of owners ...
1,0100000US,NaN,United States,NaN,00,Total for all sectors,NaN,00,All owners of respondent firms,001,All firms,CA,Founded or started,2016,3641879,69.7,0.2,0.3


In [7]:
# drop unneeded colums
entpre_acq_dirty_df.drop(['GEO.id2', 'GEO.annotation.id', 'NAICS.id', 'NAICS.annotation.id', 'ASECBO.id','YIBSZFI.id','ACQBUS.id'], axis=1, inplace=True)
entpre_acq_dirty_df.head(2)

,GEO.id,GEO.display-label,NAICS.display-label,ASECBO.display-label,YIBSZFI.display-label,ACQBUS.display-label,YEAR.id,OWNPDEMP,OWNPDEMP_PCT,OWNPDEMP_S,OWNPDEMP_PCT_S
0,Geographic identifier code,Geographic area name,Meaning of 2012 NAICS code,"Meaning of Gender, ethnicity, race, and vetera...",Meaning of Years in business code,Meaning of How the owner initially acquired th...,Year,Number of owners of respondent firms with paid...,Percent of number of owners of respondent firm...,Relative standard error of number of owners of...,Standard error of percent of number of owners ...
1,0100000US,United States,Total for all sectors,All owners of respondent firms,All firms,Founded or started,2016,3641879,69.7,0.2,0.3


In [8]:
# rename columns using first row
entpre_acq_dirty_df.columns = [get_cel_value(entpre_acq_dirty_df, x) for x in entpre_acq_dirty_df.columns]
# remove first row
entpre_acq_dirty_df.drop(entpre_acq_dirty_df.index[0], inplace=True)
# remove columns with empty cells
entpre_acq_dirty_df.dropna(axis=1, how='all', inplace=True)
entpre_acq_dirty_df.head(5)

,Geographic identifier code,Geographic area name,Meaning of 2012 NAICS code,"Meaning of Gender, ethnicity, race, and veteran status code",Meaning of Years in business code,Meaning of How the owner initially acquired the business code,Year,Number of owners of respondent firms with paid employees,Percent of number of owners of respondent firms with paid employees (%),Relative standard error of number of owners of respondent firms with paid employees (%),Standard error of percent of number of owners of respondent firms with paid employees (%)
1,0100000US,United States,Total for all sectors,All owners of respondent firms,All firms,Founded or started,2016,3641879,69.7,0.2,0.3
2,0100000US,United States,Total for all sectors,All owners of respondent firms,All firms,Purchased,2016,1114797,21.3,0.6,0.2
3,0100000US,United States,Total for all sectors,All owners of respondent firms,All firms,Inherited,2016,215576,4.1,1.2,0.1
4,0100000US,United States,Total for all sectors,All owners of respondent firms,All firms,Transfer of ownership or gift,2016,380986,7.3,1.4,0.1
5,0100000US,United States,Total for all sectors,All owners of respondent firms,All firms,Total reporting,2016,5223412,100.0,0.1,0.0


In [9]:
for col_name in entpre_acq_dirty_df.columns:
    print(col_name)

Geographic identifier code
Geographic area name
Meaning of 2012 NAICS code
Meaning of Gender, ethnicity, race, and veteran status code
Meaning of Years in business code
Meaning of How the owner initially acquired the business code
Year
Number of owners of respondent firms with paid employees 
Percent of number of owners of respondent firms with paid employees (%) 
Relative standard error of number of owners of respondent firms with paid employees (%) 
Standard error of percent of number of owners of respondent firms with paid employees (%) 


In [10]:
#separating column names of integer and characters
#character columns
df_char = entpre_acq_dirty_df.iloc[:,0:5]
# changing column names to have no spaces, reduce extra words, and reduce larger words with smaller ones
df_char.columns = [c.lower()
                   .replace('geographic','geo')
                   .replace('meaning of', '')
                   .replace('and','')
                   .replace('identifier','id')
                   .replace(' ', '_')
                   .replace(',', '_')
                   for c in df_char.columns]
df_char.dtypes

geo_id_code                                       object
geo_area_name                                     object
_2012_naics_code                                  object
_gender__ethnicity__race___veteran_status_code    object
_years_in_business_code                           object
dtype: object

In [11]:
#separating column names of integer and characters
#character columns
df_char =  entpre_acq_dirty_df.iloc[:,0:5]
# changing column names to have no spaces, reduce extra words, and reduce larger words with smaller ones
df_char.columns = [c.lower()
                   .replace('geographic','geo')
                   .replace('meaning of', '')
                   .replace('and','')
                   .replace('identifier','id')
                   .replace(' ', '_')
                   .replace(',', '_')
                   for c in df_char.columns]
#getting list of column names
cats = []
for col in df_char.columns:
    cats.append(col)

#passing list through dataframe to make them categorical
df_char[cats] = df_char[cats].astype('category')

#.cat.codes can only work on series, going through each column and making into codes

for col in df_char:
    code = col + '1'
    df_char[code] = df_char[col].cat.codes
df_char.drop(columns = 'geo_id_code1')
df_char.dtypes

C:\Users\adfil\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\adfil\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


geo_id_code                                        category
geo_area_name                                      category
_2012_naics_code                                   category
_gender__ethnicity__race___veteran_status_code     category
_years_in_business_code                            category
geo_id_code1                                           int8
geo_area_name1                                         int8
_2012_naics_code1                                      int8
_gender__ethnicity__race___veteran_status_code1        int8
_years_in_business_code1                               int8
dtype: object

In [12]:
#integer columns
df_int = entpre_acq_dirty_df.iloc[:,6:10]

# changing column names to have no spaces, reduce extra words, and reduce larger words with smaller ones
df_int.columns = [c.lower()
                  .replace('(','')
                  .replace(')','')
                  .replace('number','num')
                  .replace('percent','per')
                  .replace('%','per')
                  .replace('integer','int')
                  .replace('relative','')
                  .replace('standard error','se')
                  .replace('of','')
                  .replace('and','')
                  .replace('with','w')
                  .replace('employees','emp')
                  .replace(' ', '_')
                  .replace(',', '')
                 
                 for c in df_int.columns]

df_float = df_int.astype(float)
df_float.head(1)

,year,num__owners__respondent_firms_w_paid_emp_,per__num__owners__respondent_firms_w_paid_emp_per_,_se__num__owners__respondent_firms_w_paid_emp_per_
1,2016.0,3641879.0,69.7,0.2


In [13]:
#printing column names with formats for SQL
for col_name in df_char.columns:
    print(f'{col_name} VARCHAR(100),')
    
for col_name in df_int.columns:
    print(f'{col_name} NUMERIC,')

geo_id_code VARCHAR(100),
geo_area_name VARCHAR(100),
_2012_naics_code VARCHAR(100),
_gender__ethnicity__race___veteran_status_code VARCHAR(100),
_years_in_business_code VARCHAR(100),
geo_id_code1 VARCHAR(100),
geo_area_name1 VARCHAR(100),
_2012_naics_code1 VARCHAR(100),
_gender__ethnicity__race___veteran_status_code1 VARCHAR(100),
_years_in_business_code1 VARCHAR(100),
year NUMERIC,
num__owners__respondent_firms_w_paid_emp_ NUMERIC,
per__num__owners__respondent_firms_w_paid_emp_per_ NUMERIC,
_se__num__owners__respondent_firms_w_paid_emp_per_ NUMERIC,


In [14]:
#concating DF back together to be imported into DB
census_final = pd.concat([df_char,df_int], axis = 1)
census_final.head()

#checking columns names are correct
for col_name in census_final.columns:
    print(col_name)

geo_id_code
geo_area_name
_2012_naics_code
_gender__ethnicity__race___veteran_status_code
_years_in_business_code
geo_id_code1
geo_area_name1
_2012_naics_code1
_gender__ethnicity__race___veteran_status_code1
_years_in_business_code1
year
num__owners__respondent_firms_w_paid_emp_
per__num__owners__respondent_firms_w_paid_emp_per_
_se__num__owners__respondent_firms_w_paid_emp_per_


In [ ]:
#verifying type
census_final.dtypes

In [16]:
#importing slim file 
census_final.to_csv("../Resources/Census_ASE1/slim_census.csv",index = False)

# file = "../Resources/Census_ASE1/slim_census.csv"
# sim_visa_db = pd.read_csv(file)

In [20]:
#turning dataframe into SQL database
conn = f"postgres:{postgres_password}@localhost:{postgres_port}/ETL_Project"
engine = create_engine(f'postgresql://{conn}')

In [21]:
census_final.to_sql(name = 'census_ASE1', con = engine, if_exists = 'append',index = False)

OperationalError: (psycopg2.OperationalError) could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 57795?
could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 57795?

(Background on this error at: http://sqlalche.me/e/e3q8)